<a href="https://colab.research.google.com/github/Kotyga/aiarrow_2024/blob/main/ml_flow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install catboost -q
!pip install eli5 -q
!pip install lime -q
!pip install shap -q

# ----------------------------------------

import warnings
warnings.filterwarnings('ignore')

# ----------------------------------------

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import KNNImputer
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_auc_score
# from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier, Pool
import eli5, lime, shap



import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
%%html
<!DOCTYPE html>
<html>
<head>
<style>
  .gradient-background {
    background: linear-gradient(to bottom right, navy, purple);
    color: white;
    height: 45px;
    display: flex;
    justify-content: center;
    align-items: center;
    font-size: 15px;
  }
</style>
</head>
<body>
  <div class="gradient-background">
      <h1> 🔥 Разбор соревнования </h1>
  </div>
</body>
</html>

В этом соревновании тебе предстоит использовать набор данных о кредитном риске. Набор данных содержит информацию о различных атрибутах клиентов. Кроме того, в наборе данных присутствует целевая переменная, которая указывает на факт дефолта по кредиту. Твоя задача заключается в построении модели, которая сможет прогнозировать вероятность возникновения дефолта на основе предоставленных атрибутов клиента.

Метрика: ROC-AUC

Описание данных

| Название характеристики | Описание |
| --- | --- |
| person_age | Возраст |
| person_income | Годовой доход |
| person_home_ownership | Владение жильем |
| person_emp_length | Продолжительность работы (в годах) |
| loan_intent | Цель кредитования |
| loan_grade | Класс кредита |
| loan_amnt | Сумма кредита |
| loan_int_rate | Процентная ставка |
| loan_status | Статус кредита (0 - не дефолт 1 - дефолт) |
| loan_percent_income | Процентный доход |
| cb_person_default_on_file | Исторический дефолт |
| cb_preson_cred_hist_length | Продолжительность кредитной истории |

Вам нужно будет предсказать loan_status.

[Автор разбора: Майя Котыга](https://www.kaggle.com/mayyakotyga)

In [ ]:
%%html
<!DOCTYPE html>
<html>
<head>
</head>
<body>
  <div class="gradient-background">
      <h2> 🔎 Обзор данных </h2>
  </div>
</body>
</html>

Препроцессинг данных играет ключевую роль в успешной работе с моделями машинного обучения. Работа с пропущенными значениями, категориальными признаками и другими аспектами данных помогает улучшить качество моделей и сделать предсказания более точными и надежными.

1. Работа с пропущенными значениями: Пропущенные данные могут исказить результаты моделирования, поэтому важно разработать стратегию их обработки. Это может включать заполнение пропущенных значений средним или медианой, удаление строк или столбцов с отсутствующими данными, или использование специальных алгоритмов для заполнения пропусков.

2. Кодирование категориальных признаков: Модели машинного обучения требуют числовых данных, поэтому необходимо преобразовать категориальные признаки в числовой формат. Это может быть достигнуто с помощью методов кодирования, таких как One-Hot Encoding или Label Encoding.

3. Масштабирование данных: Некоторые модели машинного обучения, например, линейная регрессия или метод ближайших соседей, требуют масштабирования данных для более эффективной работы. Это может быть выполнено с использованием стандартизации или нормализации данных.

В целом, хороший препроцессинг данных позволяет улучшить качество моделей машинного обучения, повысить их эффективность и точность прогнозов, а также уменьшить вероятность переобучения модели.

In [ ]:
df = pd.read_csv('/kaggle/input/predictingtheprobabilityofloandefault/train.csv')
df.head()

In [ ]:
df.info()

## Работа с категориальными признаками
### Стратегии
* Ничего не делать (работает с немногими моделями)
* OHE
* Label Encoding
* Frequency Encodind
* Выкинуть (можем выкинуть хороший признак)

In [ ]:
l_e = []
for i in df.columns:
    if df[i].dtype == 'O':
        m = LabelEncoder()
        df[i] = m.fit_transform(df[i])
        l_e.append(m)

## Работа с пропущенными значениями

### Стратегии

* Выбросить
* Заполнить нулями
* Заполнить статситикой (среднее, медиана, мода)
* Посмотреть на соседа

In [ ]:
round(df.isna().sum() / df.shape[0] * 100, 2)

#### KNNImputer
![image.png](attachment:a3a2b45e-1efa-4d5c-8b78-5d8b26e0c28d.png)

In [ ]:
imputer = KNNImputer(n_neighbors=25)
df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)
df.head()

## Работа с разномасштабными данными
### Стратегии
* Ничего (проклятье размерности)
* BoxCox преобразование
* StandardScaler

In [ ]:
# scaler = StandardScaler()
# df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [ ]:
%%html
<!DOCTYPE html>
<html>
<head>
</head>
<body>
  <div class="gradient-background">
      <h2> 📊 Визуальный анализ </h2>
  </div>
</body>
</html>

Визуальный анализ графиков является важным инструментом для участников соревнований по Data Science, так как он позволяет лучше понять данные, выделить закономерности и тренды, а также выявить аномалии и выбросы. Визуализация данных помогает более наглядно представить информацию и делает процесс анализа более эффективным и понятным. Кроме того, графики могут помочь визуально сравнить различные модели или подходы к анализу данных, что помогает выбрать наилучшее решение. В целом, визуальный анализ является важным этапом работы над данными и помогает улучшить качество и точность результатов.

In [ ]:
sns.pairplot(
    df,
    corner=True,
    hue="loan_status",
);

In [ ]:
c_1 = pd.DataFrame(df['loan_status'].value_counts())

plt.pie(c_1['count'],
         labels = ['Не дефолт', 'Дефолт'],
         autopct='%1.1f%%')
plt.title('Доля дефолтных кредитов, %');

In [ ]:
corr = df.corr(numeric_only=True).round(2)
corr.style.background_gradient(cmap="RdYlGn")

In [ ]:
sns.catplot(
    data=df,
    x="loan_grade",
    y="loan_amnt",
    hue="loan_status",
    aspect=4,
    kind="boxen",
).set_xticklabels(rotation=45, horizontalalignment="right");

In [ ]:
%%html
<!DOCTYPE html>
<html>
<head>
</head>
<body>
  <div class="gradient-background">
      <h2> 🗑️ Работа с выбросами </h2>
  </div>
</body>
</html>

Работа с выбросами важна для обеспечения корректной работы модели на данных. Выбросы - это значения, которые сильно отличаются от основной массы данных и могут привести к искажению результатов анализа. Отфильтровать выбросы необходимо для того, чтобы модель машинного обучения не переобучилась на них и не потеряла способность обобщать данные.

Однако при этом важно не удалять все выбросы, так как они могут содержать важную информацию или отражать особенности данных. Выбросы в некоторых случаях могут быть результатом ошибок в данных или нестандартных ситуаций, которые также могут произойти в тестовом датасете. Поэтому сбалансированный подход к работе с выбросами позволит достичь оптимального качества модели и ее способности к обобщению на новые данные.

#### Некоторые алгоритмы
* DBSCAN
* Правило 3 сигм (для нормального распредедния)
* Статистические тесты
* Метод перцентилей
* Isolation Forest

#### Isolation Forest

![](https://i2.wp.com/miro.medium.com/0*0GuMixLdSZo3V3Nh.)

In [ ]:
clf = IsolationForest(contamination=0.05)
clf.fit(df)
y_pred = clf.predict(df)

df = df[y_pred == 1]

In [ ]:
df.shape

In [ ]:
%%html
<!DOCTYPE html>
<html>
<head>
</head>
<body>
  <div class="gradient-background">
      <h2> 🤖 Построение моделей </h2>
  </div>
</body>
</html>

#### Catboost
![](https://newtechaudit.ru/wp-content/uploads/2021/12/1456.jpg)

![](https://scikit-learn.ru/wp-content/uploads/2021/10/image-161.png)

In [ ]:
X = df.drop("loan_status", axis=1, errors="ignore")
y = df[["loan_status"]]

n_splits = 10
clfs = []
scores = []

kf = KFold(n_splits=n_splits, shuffle=True, random_state=7575)
for num, (train_index, test_index) in enumerate(kf.split(X)):

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    clf = CatBoostClassifier(
        iterations=2000,
        depth=6,
        learning_rate=0.1,
        l2_leaf_reg=4,
        cat_features=[],
        thread_count=-1,
        random_seed=7575,
        class_weights=[1, 1],
        leaf_estimation_method='Newton',
        subsample=0.60,
        verbose=False
    )

    clfs.append(clf)

    clf.fit(X_train, y_train["loan_status"])

    y_pred = clf.predict(X_test)
    score = np.mean(np.array(y_pred == y_test["loan_status"]))
    scores.append(score)
    print(f"fold: {num} acc: {score}")

assert len(clfs) == n_splits  # Проверка, что все ок

# Считаем среднее и дисперсию по всем фолдам
print("mean accuracy score --", np.mean(scores, dtype="float16"), np.std(scores).round(4))

In [ ]:
clfs = []
scores = []
kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=7575)

for train_index, test_index in kf.split(X=X, y=y["loan_status"]):

    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # print(X_train.shape, y_train.shape)
    clf = CatBoostClassifier(
        iterations=2000,
        depth=6,
        learning_rate=0.1,
        l2_leaf_reg=4,
        cat_features=[],  # список категориальных признаков, если есть
        thread_count=-1,
        random_seed=7575,
        class_weights=[1, 1],  # балансировка классов
        leaf_estimation_method='Newton',
        subsample=0.60,
        verbose=False
    )

    clfs.append(clf)

    clf.fit(X_train, y_train["loan_status"])

    y_pred = clf.predict(X_test)
    score = np.mean(np.array(y_pred == y_test["loan_status"]))
    scores.append(score)
    print(f"fold: acc: {score}")

assert len(clfs) == n_splits
print(
    "mean accuracy score --", np.mean(scores, dtype="float16"), np.std(scores).round(4)
)

In [ ]:
%%html
<!DOCTYPE html>
<html>
<head>
</head>
<body>
  <div class="gradient-background">
      <h2> 🎯 Предсказываем на тесте </h2>
  </div>
</body>
</html>

In [ ]:
test = pd.read_csv('/kaggle/input/predictingtheprobabilityofloandefault/test.csv')
test.info()

In [ ]:
l_e_t = []
for i in test.columns:
    if test[i].dtype == 'O':
        m = LabelEncoder()
        test[i] = m.fit_transform(test[i])
        l_e_t.append(m)

In [ ]:
imputer = KNNImputer(n_neighbors=25)
test = pd.DataFrame(imputer.fit_transform(test), columns=test.columns)

In [ ]:
sub = pd.read_csv('/kaggle/input/predictingtheprobabilityofloandefault/sample_submission.csv')

In [ ]:
targets = ['loan_status']

In [ ]:
# массив для записи финального прогноза size*n_class
y_pred = np.zeros((sub.shape[0], df[targets].nunique()[0]))

# Используем все модели из списка clfs для инференса
for n, clf in enumerate(clfs):
    y_pred += clf.predict_proba(test)

In [ ]:
f = lambda x: clf.classes_[x]
sub["prediction"] = list(map(f, y_pred.argmax(axis=1)))

sub.to_csv("catboost_kfold_v1.csv", index=False)

Для того, чтобы много раз не делать одни и те же шаги, используйте функции и pipeline

In [ ]:
%%html
<!DOCTYPE html>
<html>
<head>
</head>
<body>
  <div class="gradient-background">
      <h2> 🤔 Интерпретация результатов </h2>
  </div>
</body>
</html>

In [ ]:
model = clfs[0]

In [ ]:
y_pred_proba = model.predict_proba(X_test)[:, 1]
print("ROC-AUC:", roc_auc_score(y_test, y_pred_proba))

In [ ]:
# your code here
# Get feature importances
feature_importances = model.get_feature_importance(data=Pool(X_train, label=y_train), type='FeatureImportance')

# Get feature names
feature_names = X_train.columns

# Sort feature importances in descending order
sorted_indices = np.argsort(feature_importances)[::-1]
sorted_feature_importances = feature_importances[sorted_indices]
sorted_feature_names = feature_names[sorted_indices]

# Plot feature importances
plt.figure(figsize=(10, 6))
plt.bar(range(len(feature_names)), sorted_feature_importances)
plt.xticks(range(len(feature_names)), sorted_feature_names, rotation=90)
plt.xlabel("Features")
plt.ylabel("Information Gain")
plt.title("Feature Importance Based on Information Gain")
plt.tight_layout()
plt.show()

In [ ]:
perm = eli5.sklearn.PermutationImportance(model, random_state=42).fit(X_train, y_train)
eli5.show_weights(perm, feature_names = X_train.columns.tolist())

In [ ]:
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values, X_test)

In [ ]:
shap.initjs()

print('Correct answer:', y_test.iloc[5])
shap.force_plot(explainer.expected_value, shap_values[5,:], X_test.iloc[5,:])

In [ ]:
explainer = lime.lime_tabular.LimeTabularExplainer(X_test.values, feature_names=X_test.columns.values.tolist(), mode='regression')
explanation = explainer.explain_instance(X_test.values[5], model.predict, num_features=5)

explanation.show_in_notebook(show_table=True)